In [1]:
# %matplotlib inline
# %load_ext watermark
import pandas as pd
from pandas import DataFrame, Series
from pandas.io import sql
import japandas as jpd
from pandas.tseries.offsets import CustomBusinessDay, CustomBusinessMonthEnd

import numpy as np
import pymysql
from datetime import datetime, timedelta
from decimal import *
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode()
from decimal import *
import re
from dateutil.parser import parse
from IPython.display import display, HTML
import json
print(pd.__version__)

figsize=(20, 8)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


0.17.1


In [2]:
with open('config.json') as f:
    conf = json.load(f)

con = pymysql.connect(host=conf['host'], user=conf['user'], passwd=conf['password'], port=conf['port']) #, database=conf['database'])

In [3]:
broker_votes = pd.read_sql("""
SELECT d.objValue AS region, IF(lastName='Nakamura','Trading',IF(lastName='Wan','Borrow',lastName)) AS analyst,c.objValue AS broker,analystVote/100 AS vote
        FROM
        hkg02p.t08BrokerVote a,hkg02p.t01Person b, hkg02p.t10ObjectText c, hkg02p.t10ObjectText d
        WHERE
        brokerVoteDate='2016-12-01'
        AND a.personID=b.personID
        AND a.brokerVoteBrokerID=c.objectTextID
        AND a.regionID = d.objectTextID
        AND analystVote>0
        ORDER BY lastName,regionID,broker
""", con)

### Vote budget

In [4]:
vote_budget_data = {
    'analyst': ['Arigami', 'Borrow', 'Forday', 'Isozaki', 'Meguro', 'Phillips', 
                'Yamashita', 'Qi', 'Nishimi', 'Shima', 'Togo', 'Trading', 'Lu', 
                'Wakahara', 'Jo', 'Ohno', 'Esparza', 'Lian', 
               ],
    'jp': [100, 50, 100, 100, 100, 100, 50, 0, 100, 100, 100, 60, 0, 100, 100, 100, 40, 0],
    'asia': [0, 25, 0, 0, 0, 0, 25, 50, 50, 50, 0, 25, 50, 0, 0, 0, 0, 50]
}

vote_budget = pd.DataFrame(vote_budget_data)

In [5]:
vote_budget

,analyst,asia,jp
0,Arigami,0,100
1,Borrow,25,50
2,Forday,0,100
3,Isozaki,0,100
4,Meguro,0,100
5,Phillips,0,100
6,Yamashita,25,50
7,Qi,50,0
8,Nishimi,50,100
9,Shima,50,100


### Credit for Type of trades

In [6]:
trade_credit = pd.DataFrame(
    {
        'type': ['Placement', 'IPO', 'Trade', 'CB'],
        'percent': [0.05, 0.1, 1.0, 0.02]
    }
)
trade_credit

,percent,type
0,0.05,Placement
1,0.10,IPO
2,1.00,Trade
3,0.02,CB


### PL target jpy

Avg assets (in USD) * 0.26 / 4 quarters

### Quarter PL

In [10]:
quarter_pl = pd.read_sql("""
SELECT a.name, a.quick, a.side, SUM(a.attribution) AS PL
FROM hkg02p.t05PortfolioResponsibilities a
WHERE
  a.processDate >= '2016-12-01' AND a.processDate <= '2017-02-28'
GROUP BY a.instrumentID, a.side
ORDER BY  SUM(a.attribution) DESC
""", con)
quarter_pl.head()

,name,quick,side,PL
0,SOFTBANK GROUP CORP,9984,L,835196088
1,DENA CO LTD,2432,S,502086130
2,KYUSHU RAILWAY COMPANY,9142,L,473150079
3,TOKYO STEEL MFG CO LTD,5423,L,443723634
4,TOSHIBA CORP,6502,S,423747289


 ### Broker evaluation from Adv

In [11]:
pl_evaluation = pd.DataFrame({
    'year': [2017]*5,
    'quarter': [1]*5,
    'quick': ['9984', '9984', '9142', '5423', '5423'],
    'broker_id': [
        18,  # MUFJ
        22,  # CITI
        10,  # Mizuho
        6,  # Daiwa
        22
    ],
    'percent': [0.1, 0.1, 0.05, 0.05, 0.05]
})
pl_evaluation


,broker_id,percent,quarter,quick,year
0,18,0.10,1,9984,2017
1,22,0.10,1,9984,2017
2,10,0.05,1,9142,2017
3,6,0.05,1,5423,2017
4,22,0.05,1,5423,2017


### Percent used

In [8]:
broker_votes.groupby(['region', 'analyst']).sum()

vote
region analyst         
Asia   Borrow     1.000
       Lian       1.000
       Lu         1.000
       Nishimi    0.330
       Qi         1.000
       Shima      0.970
       Trading    1.000
       Yamashita  0.300
Japan  Arigami    0.937
       Borrow     1.000
       Esparza    1.000
       Forday     0.455
       Isozaki    0.961
       Jo         0.975
       Meguro     1.000
       Nishimi    1.000
       Ohno       0.990
       Phillips   0.800
       Shima      1.000
       Togo       0.840
       Trading    1.010
       Wakahara   0.985
       Yamashita  1.000